# 自动求导

**自动求导（autodiff）中的两种主要模式** 
—— 正向累积（forward accumulation）和反向累积（reverse accumulation，也叫反向传播），
这是现代深度学习框架（如 PyTorch、TensorFlow）实现梯度计算的核心原理之一。

下面我们**按层级**详细讲解：

---

## 📌 1. 链式法则（Chain Rule）

$$
\frac{\partial y}{\partial x} = \frac{\partial y}{\partial u_n} \cdot \frac{\partial u_n}{\partial u_{n-1}} \cdot \dots \cdot \frac{\partial u_2}{\partial u_1} \cdot \frac{\partial u_1}{\partial x}
$$

这是最基本的**链式求导规则**，适用于变量经过多层函数嵌套的情形：

例如：
- $x \rightarrow u_1 = f_1(x)$
- $u_1 \rightarrow u_2 = f_2(u_1)$
- ...
- $u_n \rightarrow y = f_{n+1}(u_n)$

那么：
- $y = f_{n+1} \circ f_n \circ \dots \circ f_1(x)$
- 对 $x$ 求导时，就必须逐层相乘所有的偏导数

---

## 📌 2. 正向累积（Forward Mode）

$$
\frac{\partial y}{\partial x} =
\frac{\partial y}{\partial u_n} \left( \frac{\partial u_n}{\partial u_{n-1}} \left( \dots \left( \frac{\partial u_2}{\partial u_1} \cdot \frac{\partial u_1}{\partial x} \right) \dots \right) \right)
$$

### ✅ 理解方式：

- **从输入开始**（从 $x$）一步一步正向地传播导数
- 每一步将当前的导数乘上下一级的偏导
- 适合：**输入变量少，输出变量多** 的情形（例如：标量输入 → 向量输出）

### 💡 应用场景：
- 科学计算中的灵敏度分析
- 编译器层的自动微分（如 JAX）

---

## 📌 3. 反向累积（Reverse Mode，也称反向传播）

$$
\frac{\partial y}{\partial x} =
\left( \left( \left( \left( \frac{\partial y}{\partial u_n} \cdot \frac{\partial u_n}{\partial u_{n-1}} \right) \cdot \dots \right) \cdot \frac{\partial u_2}{\partial u_1} \right) \cdot \frac{\partial u_1}{\partial x} \right)
$$

### ✅ 理解方式：

- 从**输出开始**（从 $y$）向后一步一步反向传播梯度
- 每一步把当前的梯度乘以上一级变量的偏导
- 适合：**输入变量多，输出变量少（尤其是标量）** 的情形

### 💡 这正是深度学习框架中最常用的“**反向传播（Backpropagation）**”原理！

---

## 🔁 比较总结

| 特点 | 正向模式 | 反向模式（反向传播） |
|------|----------|------------------------|
| 方向 | 从输入向输出 | 从输出向输入 |
| 优点 | 适合少输入多输出 | 适合多输入单输出（典型深度学习） |
| 应用 | JAX / 数值模拟 | PyTorch / TensorFlow 等神经网络框架 |
| 难点 | 存储导数信息 | 需要构建计算图和缓存中间变量 |

---

## 🧠 小贴士：

- PyTorch 默认采用 **反向传播模式**，执行 `loss.backward()` 时就是执行了这张图中最下面那一行公式。
- JAX 可以使用正向或反向模式自动微分，适合科研工程。
- 你现在看到的推导其实就是计算图上使用链式法则的不同实现方式。

---

## ✅ 总结一句话：

> 自动求导的本质就是链式法则，但方向不同（正向 or 反向），适合的应用场景也不同。

---



## 自动求导的实现

假设我们想对函数$y = 2X^TX$关于列向量$X$求导

In [2]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算$y$关于$X$的梯度之前，我们需要一个地方来存储梯度。

In [3]:
x.requires_grad_(True)  #等价于'x = torch.arange(4.0, requires_grad = True)'
x.grad  #默认时None

In [4]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

通过用反向传播函数来自动计算$y$关于$X$每个分量的梯度

In [5]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [6]:
x.grad == 4 * x

tensor([True, True, True, True])

这段代码是 PyTorch 中 **自动求导（autograd）机制的一个典型例子**，用来演示如何对一个**标量函数**进行 **反向传播（backward）**，并计算输入变量的梯度。

我们来**逐行讲解每一个细节**👇

---

## ✅ 代码回顾：

```python
import torch

x = torch.arange(4.0)  # tensor([0., 1., 2., 3.])
x.requires_grad_(True)  # 开启梯度跟踪

x.grad  # 此时为 None，因为还没进行反向传播

y = 2 * torch.dot(x, x)  # y 是一个标量（内积结果 * 2）
y.backward()  # 反向传播，计算 dy/dx
x.grad  # 查看梯度
x.grad == 4 * x  # 是否等于 4x？（验证梯度结果）
```

---

## 🧠 第一步：`x = torch.arange(4.0)`

```python
x = tensor([0., 1., 2., 3.])
```

创建一个一维张量 $\mathbf{x} = [0, 1, 2, 3]$，元素类型为 float。

---

## 🧠 第二步：`x.requires_grad_(True)`

开启 **梯度跟踪**，告诉 PyTorch：

> “以后在计算图中涉及 `x` 的运算，都要记录下来，以便之后可以反向传播求梯度。”

这时 `x.grad` 还为 `None`，因为我们**还没有执行 `backward()`**。

---

## 🧠 第三步：`y = 2 * torch.dot(x, x)`

计算内积再乘 2：

$$
y = 2 \cdot (\mathbf{x}^\top \mathbf{x}) = 2 \cdot \sum_{i=0}^{3} x_i^2
$$

用公式写是：

$$
y = 2(x_0^2 + x_1^2 + x_2^2 + x_3^2)
$$

代入实际数值：
$$
y = 2 \cdot (0^2 + 1^2 + 2^2 + 3^2) = 2 \cdot (0 + 1 + 4 + 9) = 2 \cdot 14 = 28
$$

注意：**y 是一个标量**，所以可以直接对它调用 `backward()`。

---

## 🧠 第四步：`y.backward()`

这是自动求导的核心操作，它会从标量 `y` 出发，在记录好的计算图中反向传播，计算所有 `requires_grad=True` 张量的梯度。

也就是计算：

$$
\frac{\partial y}{\partial x_i} = \frac{\partial}{\partial x_i} \left( 2 \sum x_i^2 \right) = 2 \cdot 2x_i = 4x_i
$$

---

## ✅ 第五步：`x.grad`

现在你可以查看 `x` 的梯度了：

```python
x.grad  →  tensor([ 0.,  4.,  8., 12.])
```

这正是 $4x$ 的结果！

---

## 🧠 第六步：`x.grad == 4 * x`

这是一个**逻辑验证**，我们手动计算 $4x$，然后看是否和自动求导得到的 `x.grad` 一致。

```python
4 * x → tensor([ 0.,  4.,  8., 12.])
```

所以：

```python
x.grad == 4 * x  → tensor([True, True, True, True])
```

全部为 `True`，说明 **PyTorch 的反向传播计算完全正确！**

---

## ✅ 总结小抄：

| 步骤 | 含义 |
|------|------|
| `x.requires_grad_(True)` | 开启梯度追踪 |
| `y = f(x)` | 构造计算图 |
| `y.backward()` | 自动求导（反向传播） |
| `x.grad` | 存储 $\frac{\partial y}{\partial x}$ 的结果 |
| 验证 | 和手动计算结果对比 |

---

如果你想更深入了解计算图是怎么构建的，或者我可以用图示画出这段计算的结构，也可以用 PyTorch 的 `grad` 函数手动求导来辅助理解，要试试看吗？😄

现在我们计算$X$的另外一个函数

In [7]:
# 在默认情况下，pytorch会积累梯度，我们需要清楚之前的值。
x.grad.zero_()  # '_' 表示原地操作
y = x.sum()
y.backward()
x.grad 

tensor([1., 1., 1., 1.])

深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和。

In [8]:
# 对非标量调用'backward'需要传入一个'gradient'参数，该参数指定微分函数
x.grad.zero_()  
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外。

In [10]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [11]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们任然可以计算得到变量的梯度。

In [13]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size = (), requires_grad = True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)

In [ ]:
标量，向量？